# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

# Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Start seldon-core

In [ ]:
!helm install ../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

In [ ]:
!helm install ../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true --set single_namespace=false

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador and API gateway**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

Install gRPC modules for the prediction protos.

In [ ]:
!cp ../../proto/prediction.proto ./proto
!cd ../../proto/tensorflow && make create_protos
!cp -vr ../../proto/tensorflow/tensorflow .
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

## Serve Single Model

In [ ]:
!kubectl create namespace test1

In [ ]:
!helm install ../../helm-charts/seldon-single-model --name mymodel --set oauth.key=oauth-key --set oauth.secret=oauth-secret --namespace test1

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-7cd068f -n test1

### Get predictions

In [ ]:
import sys
sys.path.append("../../notebooks")
from seldon_utils import *
API_AMBASSADOR="localhost:8003"
API_GATEWAY_REST="localhost:8002"
API_GATEWAY_GRPC="localhost:8004"

#### REST Request

In [ ]:
r = rest_request_api_gateway("oauth-key","oauth-secret","test1",API_GATEWAY_REST)
print(r.text)

In [ ]:
r = rest_request_ambassador("mymodel","test1",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("mymodel","test1",API_AMBASSADOR)

In [ ]:
grpc_request_api_gateway("oauth-key","oauth-secret","test1",rest_endpoint=API_GATEWAY_REST,grpc_endpoint=API_GATEWAY_GRPC)

In [ ]:
!helm delete mymodel --purge

## Serve AB Test

In [ ]:
!helm install ../../helm-charts/seldon-abtest --name myabtest --set oauth.key=oauth-key --set oauth.secret=oauth-secret --namespace test1

In [ ]:
!kubectl rollout status deploy/myabtest-abtest-41de5b8 -n test1
!kubectl rollout status deploy/myabtest-abtest-df66c5c -n test1

### Get predictions

In [ ]:
r = rest_request_api_gateway("oauth-key","oauth-secret","test1",API_GATEWAY_REST)
print(r.text)

In [ ]:
r = rest_request_ambassador("myabtest","test1",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("myabtest","test1",API_AMBASSADOR)

In [ ]:
grpc_request_api_gateway("oauth-key","oauth-secret","test1",rest_endpoint=API_GATEWAY_REST,grpc_endpoint=API_GATEWAY_GRPC)

In [ ]:
!helm delete myabtest --purge

## Serve Multi-Armed Bandit

In [ ]:
!helm install ../../helm-charts/seldon-mab --name mymab --set oauth.key=oauth-key --set oauth.secret=oauth-secret --namespace test1

In [ ]:
!kubectl rollout status deploy/mymab-abtest-41de5b8 -n test1
!kubectl rollout status deploy/mymab-abtest-b8038b2 -n test1
!kubectl rollout status deploy/mymab-abtest-df66c5c -n test1

### Get predictions

In [ ]:
r = rest_request_api_gateway("oauth-key","oauth-secret","test1",API_GATEWAY_REST)
print(r.text)

In [ ]:
r = rest_request_ambassador("mymab","test1",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("mymab","test1",API_AMBASSADOR)

In [ ]:
grpc_request_api_gateway("oauth-key","oauth-secret","test1",rest_endpoint=API_GATEWAY_REST,grpc_endpoint=API_GATEWAY_GRPC)

In [ ]:
!helm delete mymab --purge